In [1]:
import cv2
import numpy as np
import os

def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X,Y,query_point,k= 5):
    vals = []
    m = X.shape[0]
    for i in range(m):
        d = dist(X[i],query_point)
        vals.append((d,Y[i]))
    vals = sorted(vals)
    #val.sort(key = lambda x: x[0])
    val = vals[:k]
    val = np.array(val)
    freq = np.unique(val,return_counts = True)
    idx = np.argmax(freq[1])
    pred = freq[0][idx]
    return pred


In [2]:
datapath = './image_data/'
face_data = []
labels = []
data = {}
class_id = 0

for i in os.listdir(datapath):
    if i.endswith('.npy'):
        data_item = np.load(datapath+i)
        data[class_id] = i[:-4]
        face_data.append(data_item)
        
        target = class_id*np.ones((data_item.shape[0],))
        labels.append(target)
        class_id += 1

facedata = np.concatenate(face_data,axis = 0)
label = np.concatenate(labels,axis = 0)

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("./SnapChat Filter dataset/haarcascade_frontalface_alt.xml")

while True:
    rest,img = cap.read()
    if rest == False:
        continue
    faces = face_cascade.detectMultiScale(img,1.3,2)
    for face in faces:
        (x,y,w,h) = face
        offset = 10
        face_selection = img[y-offset:y+h+offset,x-offset:x+w+offset]
        face_selection = cv2.resize(face_selection,(100,100))
        out = knn(facedata,label,face_selection.flatten())
        
        name = data[int(out)]
        cv2.putText(img,name,(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(img,(x,y),(x+w,x+h),(0,0,255),2)
    cv2.imshow('live vedio',img)
    key = cv2.waitKey(1)&0xFF
    if key == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
